In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

from nltk.tokenize import word_tokenize
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download('punkt')

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

!pip3 install cltk
import cltk
from cltk.tokenize.sentence import TokenizeSentence
from cltk.stop.classical_hindi.stops import STOPS_LIST

!pip3 install stanza

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import stanza
stanza.download('hi')

2020-11-29 13:10:16 INFO: Downloading default packages for language: hi (Hindi)...
2020-11-29 13:10:31 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
hindi_txt = 

In [ ]:
input_path = "/content/drive/My Drive/Colab Notebooks (1)/cod/data/all_classes_data.txt"
input_path2 = "/content/drive/My Drive/Colab Notebooks (1)/cod/data/all_classes_data2.txt"
input_path3 = "/content/drive/MyDrive/Colab Notebooks (1)/cod/data/new_data_labels.csv"
stopwords_path = "/content/drive/My Drive/Colab Notebooks (1)/cod/data/stopwords.txt"

entire_data_path = "/content/drive/My Drive/Colab Notebooks (1)/cod/data/processed.csv"

In [ ]:
main_tags = ['local_news','health','education','employment', 'prices_inequality', 'industry','migration', 'infrastructure_services', 'consumer_issues', 'culture_&_entertainment', 'environment','agriculture','livelihood', 'social_issues', 'governance','community_groups' ]

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git

fatal: destination path 'fastText' already exists and is not an empty directory.


In [ ]:
%cd fastText
!pip3 install .

/content/fastText
Processing /content/fastText
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3030398 sha256=a56c7a75547de042ae1baf7ea93e1a7de84e78d72a35598227da52f8c48bc304
  Stored in directory: /tmp/pip-ephem-wheel-cache-lrj6rtzx/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext
  Found existing installation: fasttext 0.9.2
    Uninstalling fasttext-0.9.2:
      Successfully uninstalled fasttext-0.9.2


In [ ]:
import fasttext.util
import fasttext

In [ ]:
# load embeddings into memory
path_pre_trained = "/content/drive/My Drive/Colab Notebooks (1)/cod/cc.hi.300.bin"
print("Loading embeddings")
ft = fasttext.load_model(path_pre_trained)

Loading embeddings


In [ ]:
# fasttext.util.reduce_model(ft, 50)

In [ ]:
ft.get_nearest_neighbors(b'\xe0\xa4\x95\xe0\xa4\xb0\xe0\xa4\xa8\xe0\xa5\x87')

[(0.7406541705131531, 'लिए'),
 (0.7016867995262146, 'कराने'),
 (0.6950420141220093, 'करना'),
 (0.6947993040084839, 'होने'),
 (0.6805431246757507, 'करके'),
 (0.6702057123184204, 'करवाने'),
 (0.6648045182228088, 'देने'),
 (0.6638022065162659, 'करनेे'),
 (0.6585147976875305, 'करते'),
 (0.6459359526634216, 'करता')]

In [ ]:
path_classes = "/content/drive/MyDrive/Colab Notebooks (1)/cod/data/classes2/"

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
def sentence_to_vec_tfidf(s, stop_words, tokenizer, final_tf_idf, tfidf_feat, row):

    # tokenize the sentence
    # print(s)
    words = tokenizer(s)
    
    # remove stop word tokens
    words = [w for w in words if not w in stop_words]
        
    sent_vec = np.zeros(300) # as word vectors are of zero length
    print(row)
    for w in words:
        emb = ft.get_word_vector(w.encode('utf8'))
        tfidf = final_tf_idf [row, tfidf_feat.index(w)]
        sent_vec += (vec * tfidf)
        weight_sum += tfidf
            
    
    return sent_vec / np.sqrt((sent_vec ** 2).sum())

# create sentence embeddings
def create_embeddings2(sentences):
    print("Creating sentence vectors")

    tf_idf_vect = TfidfVectorizer(stop_words=None, tokenizer=word_tokenize)
    final_tf_idf = tf_idf_vect.fit_transform(sentences)
    tfidf_feat = tf_idf_vect.get_feature_names()

    vectors = []
    row = 0
    for review in sentences:
        words = word_tokenize(review)
    
        # remove stop word tokens
        words = [w for w in words if not w in STOPS_LIST]
            
        sent_vec = np.zeros(300) # as word vectors are of zero length
        # print(row)
        for w in words:
            emb = ft.get_word_vector(w.encode('utf8'))
            tfidf = final_tf_idf [row, tfidf_feat.index(w)]
            sent_vec += (emb * tfidf)                
        
        if np.sqrt((sent_vec ** 2).sum()) !=0:
            sent_vec = sent_vec / np.sqrt((sent_vec ** 2).sum())
        vectors.append(sent_vec)

        row+=1
    return vectors




def train_fasttext_SVM2(df, cls, col, k=5):


    kf = KFold(n_splits=k)
    kf.get_n_splits(df)

    X = np.array(create_embeddings2(df[col]))
    print(X.shape)

    acc = []
    i = 0

    YPred = np.array([])
    YTest = np.array([])

    row = []

    for train_index, test_index in kf.split(df):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = df[cls][train_index], df[cls][test_index]

        xtrain = X_train
        xtest = X_test

        # print(type(xtrain))
        # print(xtrain)

        # initialize logistic regression model
        model = SVC(C=10,kernel='linear')

        #scaling
        scaler = StandardScaler()
        scaler.fit(xtrain)
        xtrainT = scaler.transform(xtrain)
        xtestT = scaler.transform(xtest)

        model.fit(xtrainT, y_train)

        # make predictions on test data
        # threshold for predictions is 0.5
        preds = model.predict(xtestT)
        # calculate accuracy
        accuracy = metrics.accuracy_score(y_test, preds)
        acc.append(accuracy)

        # print(type(y_test), type(preds), preds.shape)
        # print(y_test.to_numpy().shape)

        YTest = np.concatenate([YTest,y_test.to_numpy()])
        YPred = np.concatenate([YPred,preds])

        print(f"Fold: {i}")
        print(f"Accuracy = {accuracy}")
        i+=1

        # print(xtest)
        # print(count_vec)
        # break

    # print(type(YTest))
    # print(YTest.shape)
    # print("Accuracy for {}: {}".format(cls, sum(acc)/len(acc)))
    print("Class: {}".format(cls))
    print("Accuracy score: {}".format(accuracy_score(YTest,YPred)))
    print("Precision score: {}".format(precision_score(YTest,YPred)))
    print("Recall score: {}".format(recall_score(YTest,YPred)))
    print("F1 score: {}".format(f1_score(YTest,YPred)))
    print("Confusion Matrix: {}".format(confusion_matrix(YTest,YPred)))
    print("")

    row.append(accuracy_score(YTest,YPred))
    row.append(confusion_matrix(YTest,YPred))
    row.append(precision_score(YTest,YPred))
    row.append(recall_score(YTest,YPred))
    row.append(f1_score(YTest,YPred))
    
    return row


def train_all_classes2(classes, col):

    rows = []

    for cls in classes:
        path = path_classes+cls+".csv"
        df_s = pd.read_csv(path,encoding='utf8')

        df_s = df_s[df_s[col].notnull()]
        df_s = df_s.reset_index(drop=True)

        r = train_fasttext_SVM2(df_s,cls,col)
        rows.append(r)
    
    newdf = pd.DataFrame(rows)
    return newdf

In [ ]:
newdf = train_all_classes2(main_tags,'only_hindi')

Creating sentence vectors
(9814, 300)
Fold: 0
Accuracy = 0.5150280183392766


KeyboardInterrupt: ignored

In [ ]:
path = path_classes+"education.csv"

In [ ]:
df_s = pd.read_csv(path,encoding='utf8')

df_s = df_s[df_s['only_hindi'].notnull()]
df_s = df_s.reset_index(drop=True)


In [ ]:
df_s

,Unnamed: 0,level_0,index,Unnamed: 0.1,Unnamed: 0.1.1,sentence,label,Unnamed: 3,agriculture,community_groups,consumer_issues,culture_&_entertainment,education,employment,environment,governance,health,industry,infrastructure_services,livelihood,local_news,migration,nan,prices_inequality,social_issues,processed_sen,removed_loc,remove_loc_name,removed_loc_name,stop_word_removal,only_hindi
0,0,4,4,4,5,1 किलोमीटर आगे मत सेंड कर रही हूं यहां पर इंजी...,"infrastructure_services, education,",0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1 किलोमीटर आगे मत सेंड कर रही हूं यहां पर इंजी...,1 किलोमीटर आगे मत सेंड कर रही हूं यहां पर इंजी...,NaN,1 किलोमीटर आगे मत सेंड कर रही हूं यहां पर इंजी...,1 किलोमीटर आगे मत सेंड कर रही हूं यहां इंजीनिय...,किलोमीटर आगे सेंड रही हूं यहां इंजीनियरिंग कॉल...
1,1,6,6,6,7,1 तारीख को 8:00 बजे स्कूलों में बच्चों को स्कू...,"education, health, governance, governance,",0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1 तारीख को 8:00 बजे स्कूलों में बच्चों को स्कू...,1 तारीख को 8:00 बजे स्कूलों में बच्चों को स्कू...,NaN,1 तारीख को 8:00 बजे स्कूलों में बच्चों को स्कू...,1 तारीख 8:00 बजे स्कूलों बच्चों स्कूल रखो,तारीख बजे स्कूलों बच्चों स्कूल रखो
2,2,7,7,7,8,1 दिन में 24000 छात्रों की जानकारी वीडियो ने क...,education,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1 दिन में 24000 छात्रों की जानकारी वीडियो ने क...,1 दिन में 24000 छात्रों की जानकारी वीडियो ने क...,NaN,1 दिन में 24000 छात्रों की जानकारी वीडियो ने क...,1 दिन 24000 छात्रों जानकारी वीडियो कहा समग्र आ...,दिन छात्रों जानकारी वीडियो कहा समग्र आईडी पोर्...
3,3,12,12,12,13,10 फरवरी 9 सितंबर को रिजर्वेशन सैटरडे सुपरकॉप ...,"health, governance, education,",0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,10 फरवरी 9 सितंबर को रिजर्वेशन सैटरडे सुपरकॉप ...,10 फरवरी 9 सितंबर को रिजर्वेशन सैटरडे सुपरकॉप ...,NaN,10 फरवरी 9 सितंबर को रिजर्वेशन सैटरडे सुपरकॉप ...,10 फरवरी 9 सितंबर रिजर्वेशन सैटरडे सुपरकॉप सुप...,फरवरी सितंबर रिजर्वेशन सैटरडे सुपरकॉप सुपर विल...
4,4,14,14,14,15,10 साल का कोई अच्छा मोहम्मद लोगों के सपनों को ...,"health, health, education",0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,10 साल का कोई अच्छा मोहम्मद लोगों के सपनों को ...,10 साल का कोई अच्छा मोहम्मद लोगों के सपनों को ...,NaN,10 साल का कोई अच्छा लोगों के सपनों को पूरा करत...,10 साल कोई अच्छा लोगों सपनों पूरा करती उसका बच...,साल कोई अच्छा लोगों सपनों पूरा करती उसका बच्चा...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6619,6619,14480,14615,14615,14616,मेरा नाम आनंद कुमार सिंह है मैं बसिया ग्राम बो...,"local_news, infrastructure_services,",0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,NaN,मेरा नाम आनंद कुमार सिंह है मैं बसिया ग्राम बो...,NaN,मेरा नाम सिंह है मैं बसिया ग्राम बोल रहा हूं प...,मेरा नाम सिंह मैं बसिया ग्राम बोल रहा हूं पंचा...,मेरा नाम सिंह मैं बसिया ग्राम बोल रहा हूं पंचा...
6620,6620,13683,13813,13813,13814,मां पिंटू कुमार राम युवा वाहिनी झारखंड हजारीबा...,"governance, governance",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,NaN,मां पिंटू कुमार राम युवा वाहिनी झारखंड से महिल...,NaN,मां राम युवा वाहिनी झारखंड से महिला जनता जल पर...,मां राम युवा वाहिनी झारखंड महिला जनता जल परीक्...,मां राम युवा वाहिनी झारखंड महिला जनता परीक्षण ...
6621,6621,4086,4125,4125,4126,तरुण शर्मा तरुण कुमार मोबाइल पानी चमोली जिले क...,"local_news,",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,NaN,तरुण शर्मा तरुण कुमार मोबाइल पानी जिले के मुख्...,NaN,मोबाइल पानी जिले के मुख्य मार्ग पर जमुना मोड़ ...,मोबाइल पानी जिले मुख्य मार्ग जमुना मोड़ समीप स...,मोबाइल पानी जिले मुख्य मार्ग जमुना मोड़ समीप स...
6622,6622,4338,4383,4383,4384,दूसरों से बोल रहा हूं स्टेशन में औरतों का जमीन...,"industry, governance, governance",0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,NaN,दूसरों से बोल रहा हूं स्टेशन में औरतों का जमीन...,NaN,दूसरों से बोल रहा हूं स्टेशन में औरतों का जमीन...,दूसरों बोल रहा हूं स्टेशन औरतों जमीन कब्जा करक...,दूसरों बोल रहा हूं स्टेशन औरतों जमीन कब्जा करक...


In [ ]:
sentences = df_s['only_hindi']

In [ ]:
tf_idf_vect = TfidfVectorizer(stop_words=None, tokenizer=word_tokenize)
final_tf_idf = tf_idf_vect.fit_transform(sentences)
tfidf_feat = tf_idf_vect.get_feature_names()



In [ ]:

vectors = []
row = 0
for review in sentences:
    words = word_tokenize(review)

    # remove stop word tokens
    words = [w for w in words if not w in STOPS_LIST]
        
    sent_vec = np.zeros(300) # as word vectors are of zero length
    # print(row)
    for w in words:
        emb = ft.get_word_vector(w.encode('utf8'))
        tfidf = final_tf_idf [row, tfidf_feat.index(w)]
        sent_vec += (emb * tfidf)                
    
    sent_vec = sent_vec / np.sqrt((sent_vec ** 2).sum())
    vectors.append(sent_vec)

    row+=1

In [ ]:
vectors

In [ ]:
# newdf = train_all_classes2(main_tags,'stop_word_removal')

In [ ]:
def sentence_to_vec(s, embedding_dict, stop_words, tokenizer, max_words):
    """
    Given a sentence and other information,
    this function returns embedding for the whole sentence
    :param s: sentence, string
    :param embedding_dict: dictionary word:vector
    :param stop_words: list of stop words, if any
    :param tokenizer: a tokenization function
    """
    # # convert sentence to string and lowercase it
    # words = str(s).lower()
    
    # tokenize the sentence
    # print(s)
    words = tokenizer(s)
    
    # remove stop word tokens
    words = [w for w in words if not w in stop_words]
    
    # keep only alpha-numeric tokens
    # words = [w for w in words if w.isalpha()]
    
    # initialize empty list to store embeddings
    M = []
    i = 0
    for w in words:
    # for evert word, fetch the embedding from
    # the dictionary and append to list of
    # embeddings
        emb = ft.get_word_vector(w.encode('utf8'))
        M.append(emb)
        i+=1
    
    while i<max_words:
        i+=1
        M.append([0]*300)

    # if we dont have any vectors, return zeros
    if len(M) == 0:
        return np.zeros(max_words)
    
    # convert list of embeddings to array
    M = np.array(M)
    M = np.ravel(M)
        
    # return normalized vector
    return M / np.sqrt((M ** 2).sum())




# create sentence embeddings
def create_embeddings(sentences, mx_words):
    print("Creating sentence vectors")
    vectors = []
    for review in sentences:
        vectors.append(
            sentence_to_vec(
                s = review,
                embedding_dict = ft,
                stop_words = STOPS_LIST,
                tokenizer = word_tokenize,
                max_words = mx_words,
            )
        )
    return vectors




def train_fasttext_SVM(df, cls, col, k=3):


    kf = KFold(n_splits=k)
    kf.get_n_splits(df)

    mx_words = 0
    for ind in df.index:
        sen = df[col][ind]
        sen = word_tokenize(sen)
        mx_words = max(len(sen),mx_words)
    print("Max_words:", mx_words)

    X = np.array(create_embeddings(df[col], mx_words))
    print(X.shape)

    acc = []
    i = 0

    YPred = np.array([])
    YTest = np.array([])

    row = []

    for train_index, test_index in kf.split(df):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = df[cls][train_index], df[cls][test_index]

        xtrain = X_train
        xtest = X_test

        # print(type(xtrain))
        # print(xtrain)

        # initialize logistic regression model
        model = SVC(C=10,kernel='linear')

        #scaling
        scaler = StandardScaler()
        scaler.fit(xtrain)
        xtrainT = scaler.transform(xtrain)
        xtestT = scaler.transform(xtest)

        model.fit(xtrainT, y_train)

        # make predictions on test data
        # threshold for predictions is 0.5
        preds = model.predict(xtestT)
        # calculate accuracy
        accuracy = metrics.accuracy_score(y_test, preds)
        acc.append(accuracy)

        # print(type(y_test), type(preds), preds.shape)
        # print(y_test.to_numpy().shape)

        YTest = np.concatenate([YTest,y_test.to_numpy()])
        YPred = np.concatenate([YPred,preds])

        # print(f"Fold: {i}")
        # print(f"Accuracy = {accuracy}")
        i+=1

        # print(xtest)
        # print(count_vec)
        # break

    # print(type(YTest))
    # print(YTest.shape)
    # print("Accuracy for {}: {}".format(cls, sum(acc)/len(acc)))
    print("Class: {}".format(cls))
    print("Accuracy score: {}".format(accuracy_score(YTest,YPred)))
    print("Precision score: {}".format(precision_score(YTest,YPred)))
    print("Recall score: {}".format(recall_score(YTest,YPred)))
    print("F1 score: {}".format(f1_score(YTest,YPred)))
    print("Confusion Matrix: {}".format(confusion_matrix(YTest,YPred)))
    print("")

    row.append(accuracy_score(YTest,YPred))
    row.append(confusion_matrix(YTest,YPred))
    row.append(precision_score(YTest,YPred))
    row.append(recall_score(YTest,YPred))
    row.append(f1_score(YTest,YPred))
    
    return row


def train_all_classes(classes, col):

    rows = []

    for cls in classes:
        path = path_classes+cls+".csv"
        df_s = pd.read_csv(path,encoding='utf8')

        df_s = df_s[df_s[col].notnull()]
        df_s = df_s.reset_index(drop=True)

        r = train_fasttext_SVM(df_s,cls,col)
        rows.append(r)
    
    newdf = pd.DataFrame(rows)
    return newdf

In [ ]:
def sentence_to_vec(s, embedding_dict, stop_words, tokenizer):
    """
    Given a sentence and other information,
    this function returns embedding for the whole sentence
    :param s: sentence, string
    :param embedding_dict: dictionary word:vector
    :param stop_words: list of stop words, if any
    :param tokenizer: a tokenization function
    """
    # # convert sentence to string and lowercase it
    # words = str(s).lower()
    
    # tokenize the sentence
    # print(s)
    words = tokenizer(s)
    
    # remove stop word tokens
    words = [w for w in words if not w in stop_words]
    
    # keep only alpha-numeric tokens
    # words = [w for w in words if w.isalpha()]
    
    # initialize empty list to store embeddings
    M = []
    for w in words:
    # for evert word, fetch the embedding from
    # the dictionary and append to list of
    # embeddings
        emb = ft.get_word_vector(w.encode('utf8'))
        M.append(emb)
            
    # if we dont have any vectors, return zeros
    if len(M) == 0:
        return np.zeros(300)
    
    # convert list of embeddings to array
    M = np.array(M)
    
    # calculate sum over axis=0
    v = M.sum(axis=0)
    
    # return normalized vector
    return v / np.sqrt((v ** 2).sum())




# create sentence embeddings
def create_embeddings(sentences):
    print("Creating sentence vectors")
    vectors = []
    for review in sentences:
        vectors.append(
            sentence_to_vec(
                s = review,
                embedding_dict = ft,
                stop_words = STOPS_LIST,
                tokenizer = word_tokenize,
            )
        )
    return vectors




def train_fasttext_SVM(df, cls, col, k=3):


    kf = KFold(n_splits=k)
    kf.get_n_splits(df)

    X = np.array(create_embeddings(df[col]))
    print(X.shape)

    acc = []
    i = 0

    YPred = np.array([])
    YTest = np.array([])

    row = []

    for train_index, test_index in kf.split(df):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = df[cls][train_index], df[cls][test_index]

        xtrain = X_train
        xtest = X_test

        # print(type(xtrain))
        # print(xtrain)

        # initialize logistic regression model
        model = SVC(C=1,kernel='linear')

        #scaling
        scaler = StandardScaler()
        scaler.fit(xtrain)
        xtrainT = scaler.transform(xtrain)
        xtestT = scaler.transform(xtest)

        model.fit(xtrainT, y_train)

        # make predictions on test data
        # threshold for predictions is 0.5
        preds = model.predict(xtestT)
        # calculate accuracy
        accuracy = metrics.accuracy_score(y_test, preds)
        acc.append(accuracy)

        # print(type(y_test), type(preds), preds.shape)
        # print(y_test.to_numpy().shape)

        YTest = np.concatenate([YTest,y_test.to_numpy()])
        YPred = np.concatenate([YPred,preds])

        # print(f"Fold: {i}")
        # print(f"Accuracy = {accuracy}")
        i+=1

        # print(xtest)
        # print(count_vec)
        # break

    # print(type(YTest))
    # print(YTest.shape)
    # print("Accuracy for {}: {}".format(cls, sum(acc)/len(acc)))
    print("Class: {}".format(cls))
    print("Accuracy score: {}".format(accuracy_score(YTest,YPred)))
    print("Precision score: {}".format(precision_score(YTest,YPred)))
    print("Recall score: {}".format(recall_score(YTest,YPred)))
    print("F1 score: {}".format(f1_score(YTest,YPred)))
    print("Confusion Matrix: {}".format(confusion_matrix(YTest,YPred)))
    print("")

    row.append(accuracy_score(YTest,YPred))
    row.append(confusion_matrix(YTest,YPred))
    row.append(precision_score(YTest,YPred))
    row.append(recall_score(YTest,YPred))
    row.append(f1_score(YTest,YPred))
    
    return row


def train_all_classes(classes, col):

    rows = []

    for cls in classes:
        path = path_classes+cls+".csv"
        df_s = pd.read_csv(path,encoding='utf8')

        df_s = df_s[df_s[col].notnull()]
        df_s = df_s.reset_index(drop=True)

        r = train_fasttext_SVM(df_s,cls,col)
        rows.append(r)
    
    newdf = pd.DataFrame(rows)
    return newdf

In [ ]:
newdf = train_all_classes(main_tags,'only_hindi')

Creating sentence vectors
(9814, 300)
Class: local_news
Accuracy score: 0.43152639087018546
Precision score: 0.42869269949066213
Recall score: 0.41165681679233745
F1 score: 0.4200020792182139
Confusion Matrix: [[2215 2692]
 [2887 2020]]

Creating sentence vectors
(10296, 300)
Class: health
Accuracy score: 0.47387334887334887
Precision score: 0.4728994559742091
Recall score: 0.4559052059052059
F1 score: 0.46424685985560277
Confusion Matrix: [[2532 2616]
 [2801 2347]]

Creating sentence vectors
(6624, 300)
Class: education
Accuracy score: 0.6431159420289855
Precision score: 0.6509554140127388
Recall score: 0.6171497584541062
F1 score: 0.6336019838809671
Confusion Matrix: [[2216 1096]
 [1268 2044]]

Creating sentence vectors
(2196, 300)
Class: employment
Accuracy score: 0.6188524590163934
Precision score: 0.6143733567046451
Recall score: 0.6384335154826958
F1 score: 0.6261723983921393
Confusion Matrix: [[658 440]
 [397 701]]

Creating sentence vectors
(2170, 300)
Class: prices_inequality


In [ ]:
print(newdf)

In [ ]:
print(newdf)

In [ ]:
newdf['labels'] = main_tags
newdf = newdf.set_index('labels')
newdf